In [294]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
import numpy as np

In [295]:
df = pd.read_csv("../data/acidentes.txt")

In [296]:
df.head()

,data_inversa,fase_dia,horario,uf,br,causa_acidente,tipo_acidente,uso_solo,pessoas,mortos,feridos_leves,feridos_graves,ilesos,feridos,classificacao_acidente,regiao
0,2018-01-01,Pleno dia,06:00:00,TO,BR-153,Falta de Atenção à Condução,Colisão frontal,Rural,2,0,0,1,1,1,Com Vítimas Feridas,Norte
1,2018-01-01,Pleno dia,08:30:00,SE,BR-101,Não guardar distância de segurança,Colisão traseira,Rural,2,0,0,0,2,0,Sem Vítimas,Nordeste
2,2018-01-01,Pleno dia,08:50:00,TO,BR-153,Falta de Atenção à Condução,Colisão lateral,Rural,3,0,0,0,2,0,NaN,Norte
3,2018-01-01,Amanhecer,05:30:00,PA,BR-155,Ingestão de Álcool,Colisão lateral,Rural,2,1,0,0,1,0,Com Vítimas Fatais,Norte
4,2018-01-01,Pleno dia,10:25:00,BA,BR-116,Ultrapassagem Indevida,Saída de leito carroçável,Rural,2,0,0,1,0,1,Com Vítimas Feridas,Nordeste


In [297]:
df = df[df['classificacao_acidente'].notna()].copy()

X = df.drop(columns=['classificacao_acidente'])
y = df['classificacao_acidente']

### **XGboost**

Escolhemos o XGBoost, pois é um dos algoritmos mais eficientes para classificação, utilizando paralelismo e otimização de recursos. Além disso, ele é muito eficiente para lidar com dados desbalanceados, encontrando padrões e relações entre as variáveis, possibilitando uma melhor performance do modelo.

In [298]:
df['classificacao_acidente'].unique()

array(['Com Vítimas Feridas', 'Sem Vítimas', 'Com Vítimas Fatais'],
      dtype=object)

In [299]:
categorical_features = X.select_dtypes(include="object").columns
numerical_features = X.select_dtypes(exclude="object").columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

O label_encoder é utilizado para transformar as classes categóricas da variável alvo (y) em valores numéricos.
Isso é necessário porque muitos algoritmos de machine learning, incluindo o XGBoost, exigem que os rótulos estejam em formato numérico para realizar a classificação.

In [300]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
print(np.unique(y_encoded))
print(label_encoder.classes_)

[0 1 2]
['Com Vítimas Fatais' 'Com Vítimas Feridas' 'Sem Vítimas']


In [301]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.33, random_state=41)

In [302]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', xgb.XGBClassifier(
        objective='multi:softmax',
        eval_metric='mlogloss'
    ))
])

In [303]:
param_grid = {
    'classifier__n_estimators': [20, 50, 100],
    'classifier__max_depth': [1,2, 3 ],
}

In [304]:
randomized_search = RandomizedSearchCV(pipeline, param_distributions=param_grid, n_iter=50, cv=5, 
                                       verbose=1, n_jobs=4, scoring='accuracy', random_state=13)
randomized_search.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


c:\Users\DellGamer\projetos\A3-USJT-Global-Cybersecurity-Threats\.venv\Lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 9 is smaller than n_iter=50. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


,estimator,"Pipeline(step...=None, ...))])"
,param_distributions,"{'classifier__max_depth': [1, 2, ...], 'classifier__n_estimators': [20, 50, ...]}"
,n_iter,50
,scoring,'accuracy'
,n_jobs,4
,refit,True
,cv,5
,verbose,1
,pre_dispatch,'2*n_jobs'
,random_state,13
,error_score,nan


In [305]:
from sklearn.metrics import (
    precision_score, recall_score, f1_score,
    confusion_matrix
)

y_pred = randomized_search.predict(X_test)
y_proba = randomized_search.predict_proba(X_test)

print(len(label_encoder.classes_))
labels = np.arange(len(label_encoder.classes_))
print(label_encoder.classes_)


print(confusion_matrix(y_test, y_pred, labels=labels))

print("Acurácia:", accuracy_score(y_test, y_pred))
print("Precisão (macro):", precision_score(y_test, y_pred, average='macro', zero_division=0))
print("Recall (macro):", recall_score(y_test, y_pred, average='macro', zero_division=0))
print("F1-score (macro):", f1_score(y_test, y_pred, average='macro', zero_division=0))

3
['Com Vítimas Fatais' 'Com Vítimas Feridas' 'Sem Vítimas']
[[ 1470     0     0]
 [    0 16453     0]
 [    0     0  4957]]
Acurácia: 1.0
Precisão (macro): 1.0
Recall (macro): 1.0
F1-score (macro): 1.0
